# CSE547 - Colab 8 
## Submodular Optimization



# Setup

Let's authenticate a Google Drive client to download the file we will be processing.

**Make sure to follow the interactive instructions.**

In [ ]:
!pip install -U -q PyDrive

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id='1VjVUIIRfxikSocPTp2FdmmLza8c-n_qC'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('book_ratings.csv')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries for our task.

In [ ]:
import pandas as pd
import numpy as np
import time

import warnings
warnings.filterwarnings("ignore")

from heapq import heappop, heappush

# Your Task

If you run successfully the setup stage, you are ready to work with the **Book-Crossing (BX) dataset**. If you want to read more about it, check the [official website](http://www2.informatik.uni-freiburg.de/~cziegler/BX/) of the dataset, which contains concise schema description of the dataset, and the download page.

In this Colab, we will use the [Pandas API](https://pandas.pydata.org/docs/user_guide/index.html) to perform submodular optimiazation algorithms on a small subset of the BX-Book-Ratings dataset (20K users). 



In [ ]:
data = pd.read_csv('book_ratings.csv')
data.head()

,userid,isbn,rating
0,276727,0446520802,0
1,276729,052165615X,3
2,276729,0521795028,6
3,276736,3257224281,8
4,276762,034544003X,0


In [ ]:
sub = data[data.userid.isin([198711, 212898, 234623])]

In [ ]:
len(sub.isbn.unique())

14324

Let's see the top 10 readers who have rated the most books, sorted in descending order. 

In [ ]:
data.groupby('userid').isbn.nunique().sort_values(ascending=False)[:10]

userid
198711    7550
212898    4785
234623    2674
102967    2352
98741     2317
190925    2154
69697     1915
78783     1879
238781    1685
23902     1503
Name: isbn, dtype: int64

In [ ]:
list(data.groupby('userid').isbn.nunique().sort_values(ascending=False)[:10].index)

[198711, 212898, 234623, 102967, 98741, 190925, 69697, 78783, 238781, 23902]

Your goal is to find the set of users who rated the most *unique* books by implementing the **greedy submodular optimization algorithm** and the **lazy greedy algorithm**. Note: A user may give a rating of 0 to a book. 


## Q1: Greedy Submodular Optimization Algorithm


We start by determining our objective function. Write a function that returns the number of unique books rated by the set of users A. 

In [ ]:
def F(A):
  # YOUR CODE HERE 
  res = []
  for i in A:
    res.append(len(data[data.userid == i]))
  return sum(res)


In [ ]:
F(["276727", "276729"])

0

In [ ]:
res = []
for i in [276727, 276727]:
  print(str(i))
  res.append(len(data[data.userid == i]))
res

276727
276727


[1, 1]

In [ ]:
users = data.userid.unique()
users

array([276727, 276729, 276736, ..., 276676, 276681, 276709])

The greedy(k) function below provides a sketch for the greedy submodular optimization algorithm where the maximum cardinality is set to be k.  Your task is to complete the function and **report the set of users and the objective values (i.e. number of unique books) for k = 2 and 3.** (They may take a while to run). You are free to modify or implement your own code.

In [ ]:
def greedy(k):

  # list of unique user id
  users = data.userid.unique()

  # initialize the list of candidate users
  greedy = []

  # initialize the time tracker
  time_k = 0

  # loop over the cardinality values
  for i in range(k):

    start_time = time.time()

    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Find the user who contributes the most gain with respect to the 
    # objective value
    # Save this user in a variable `candidate_user` 
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # YOUR CODE HERE 
    candidate_user = data.groupby('userid').isbn.nunique().sort_values(ascending=False).index
    candidate_user = list(candidate_user)[i]

    # update the list of candidate users
    greedy = greedy + [candidate_user]

    end_time = time.time() - start_time
    time_k += end_time

    print('k = {}'.format(i+1))
    print("The set of users: {}".format(greedy))
    print("Objective value = {}".format(F(greedy)))
    print("Time taken = {} seconds".format(round(time_k, 5)))

    # -------------------------------------------
    # remove the chosen candidate from user list
    #-------------------------------------------
    # YOUR CODE HERE
    #users.remove(candidate_user)

In [ ]:
greedy(7)

k = 1
The set of users: [198711]
Objective value = 7550
Time taken = 0.15303 seconds
k = 2
The set of users: [198711, 212898]
Objective value = 12335
Time taken = 0.26618 seconds
k = 3
The set of users: [198711, 212898, 234623]
Objective value = 15009
Time taken = 0.32414 seconds
k = 4
The set of users: [198711, 212898, 234623, 102967]
Objective value = 17361
Time taken = 0.386 seconds
k = 5
The set of users: [198711, 212898, 234623, 102967, 98741]
Objective value = 19678
Time taken = 0.43929 seconds
k = 6
The set of users: [198711, 212898, 234623, 102967, 98741, 190925]
Objective value = 21832
Time taken = 0.49775 seconds
k = 7
The set of users: [198711, 212898, 234623, 102967, 98741, 190925, 69697]
Objective value = 23747
Time taken = 0.55708 seconds


## Q2: Lazy Greedy Algorithm

Now, revise the code by implementing a lazy greedy optimization algorithm. **Report the set of users and the objective values for k = 7 and 8.**

*Hints:* (1) The results should be the same for all k between the two algorithms. (2) The objective value for k = 5 is 18200. (3) It may be useful, but not required, to use the [heapq](https://docs.python.org/2/library/heapq.html) algorithm in keeping the sorted array.









In [ ]:
def lazy(k): 

  # list of unique user id
  users = data.userid.unique()

  # initialize the list of candidate users
  lazy_greedy = []

  # ---------------------------------------------------------
  # You may want to initialize a list of marginal gains
  # --------------------------------------------------------
  # YOUR CODE HERE

  
  # initialize the time tracker
  time_k = 0
  
  # loop over the cardinality value
  for i in range(k):
    start_time = time.time()

    if i == 0:

      # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
      # Find the user who contributes the most gain with respect to the 
      # objective value, 
      # pop and save this user in a variable `candidate_user', 
      # keep the rest in a sorted array with repect to their marginal gains 
      # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
      # YOUR CODE HERE 
      candidate_user = 198711
      marginal_gains =list(data.groupby('userid').isbn.nunique().sort_values(ascending=False)[:20].index)

    else:  
      # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
      # This should include a while loop, where a comparison between the second
      # best candidate and the third best candidate is made. 
      # Once you find the next candidate, pop and save this user in `candidate_user' 
      # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
      # YOUR CODE HERE
      original = data[data.userid == candidate_user].isbn
      choice1 = data[data.userid == marginal_gains[0]].isbn
      choice2 = data[data.userid == marginal_gains[1]].isbn


    lazy_greedy.append(candidate_user)

    end_time = time.time() - start_time
    time_k += end_time 

    print('k = {}'.format(i+1))
    print("The set of users: {}".format(lazy_greedy))
    print("Objective value = {}".format(F(lazy_greedy)))
    print("Time taken = {} seconds".format(round(time_k, 5)))
    

In [ ]:
import heapq

def lazy(k): 

  # list of unique user id
  users = data.userid.unique()
  start_time = time.time()
  # initialize the list of candidate users
 
  gains = []
  for user in users:
      #spread = compute_independent_cascade(graph, [node], prob, n_iters)
      score = F([user])
      heapq.heappush(gains, (-score, user))

  # we pop the heap to get the node with the best spread,
  # when storing the spread to negate it again to store the actual spread
  score, user = heapq.heappop(gains)
  solution = [user]
  score = -score
  scores = [score]

  # record the number of times the spread is computed
  lookups = [len(users)]

  for i in range(k - 1):
      node_lookup = 0
      matched = False

      while not matched:
          node_lookup += 1

          # here we need to compute the marginal gain of adding the current node
          # to the solution, instead of just the gain, i.e. we need to subtract
          # the spread without adding the current node
          _, current_node = heapq.heappop(gains)
          score_gain = F(solution + [current_node]) - score

          # check if the previous top node stayed on the top after pushing
          # the marginal gain to the heap
          heapq.heappush(gains, (-score_gain, current_node))
          matched = gains[0][1] == current_node

      # spread stores the cumulative spread
      score_gain, node = heapq.heappop(gains)
      score -= score_gain
      solution.append(node)
      scores.append(score)
      lookups.append(node_lookup)


      end_time = time.time() - start_time


      print('k = {}'.format(i+1))
      print("The set of users: {}".format(solution))
      print("Objective value = {}".format(F(solution)))
      print("Time taken = {} seconds".format(round(end_time, 5)))
    

In [ ]:
lazy(7)

k = 1
The set of users: [198711, 212898]
Objective value = 12335
Time taken = 8.18648 seconds
k = 2
The set of users: [198711, 212898, 234623]
Objective value = 15009
Time taken = 8.19253 seconds
k = 3
The set of users: [198711, 212898, 234623, 102967]
Objective value = 17361
Time taken = 8.19697 seconds
k = 4
The set of users: [198711, 212898, 234623, 102967, 98741]
Objective value = 19678
Time taken = 8.20365 seconds
k = 5
The set of users: [198711, 212898, 234623, 102967, 98741, 190925]
Objective value = 21832
Time taken = 8.21047 seconds
k = 6
The set of users: [198711, 212898, 234623, 102967, 98741, 190925, 69697]
Objective value = 23747
Time taken = 8.21801 seconds


Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**!